# Exercício de Refatoração: Análise da Qualidade de Vinhos

Neste exercício, vamos refatorar o código que analisa um conjunto de dados com dados sobre a qualidade de vinhos retirado da 
[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/wine+quality). Cada linha contém dados de uma 
amostra de vinho, incluindo várias propriedades fico-químicas coletadas dos testes, assim como também avaliações dadas por
especialistas em vinhos.

O código deste notebook primeiro renomeia as colunas do conjunto de dados e depois calcula algumas estatísticas em como algumas
features podem estar relacionadas com as avaliações de qualidade. 

Este notebook foi traduzido e retirado do programa da Udacity Machine Learning Engineer Nanodegree. A atividade foi realizada e 
completa, algumas células foram apagadas para deixar o notebook mais objetivo.


In [8]:
import pandas as pd
df = pd.read_csv('winequality-red.csv', sep=';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Renomeando colunas

Para podermos referenciar as colunas com a notação de ponto (df.column_name) vamos remover os espaços em branco. Aqui está uma forma de fazer isso:

In [9]:
new_df = df.rename(columns={'fixed acidity': 'fixed_acidity',
                             'volatile acidity': 'volatile_acidity',
                             'citric acid': 'citric_acid',
                             'residual sugar': 'residual_sugar',
                             'free sulfur dioxide': 'free_sulfur_dioxide',
                             'total sulfur dioxide': 'total_sulfur_dioxide'
                            })
new_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Aqui está um modo um pouco melhor de fazer isso. É possível evitar erros de nomeação que podem ocorrer devido à digitação manual.

In [ ]:
labels = list(df.columns)
labels[0] = labels[0].replace(' ', '_')
labels[1] = labels[1].replace(' ', '_')
labels[2] = labels[2].replace(' ', '_')
labels[3] = labels[3].replace(' ', '_')
labels[5] = labels[5].replace(' ', '_')
labels[6] = labels[6].replace(' ', '_')
df.columns = labels

df.head()

O modo acima é ainda um pouco repetivo. Vamos fazer um pouco melhor:

In [12]:
labels = list(df.columns)
for i in range(0,7):
    labels[i] = labels[i].replace(' ', '_')
df.columns = labels

df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Analizando Features

Agora que nossas colunas estão prontas, precisamos ver como as diferentes features do conjunto de dados se relacionam com as avaliações dos vinhos. Um modo muito simples de fazer isso é por meio da observação da média da avaliação da qualidade para a metade melhor avaliada, assim como para a metade pior avaliada. O código abaixo faz essa avaliação para quatro features: álcool, pH, ácido cítrico e açúcar residual. Porém, o código ainda é muito repetivo, dê uma olhada:

In [14]:
data = df.copy()

In [15]:
median_alcohol = df.alcohol.median()
for i, alcohol in enumerate(df.alcohol):
    if alcohol >= median_alcohol:
        df.loc[i, 'alcohol'] = 'high'
    else:
        df.loc[i, 'alcohol'] = 'low'
df.groupby('alcohol').quality.mean()

alcohol
high    5.958904
low     5.310302
Name: quality, dtype: float64

In [16]:
median_pH = df.pH.median()
for i, pH in enumerate(df.pH):
    if pH >= median_pH:
        df.loc[i, 'pH'] = 'high'
    else:
        df.loc[i, 'pH'] = 'low'
df.groupby('pH').quality.mean()

pH
high    5.598039
low     5.675607
Name: quality, dtype: float64

In [17]:
median_sugar = df.residual_sugar.median()
for i, sugar in enumerate(df.residual_sugar):
    if sugar >= median_sugar:
        df.loc[i, 'residual_sugar'] = 'high'
    else:
        df.loc[i, 'residual_sugar'] = 'low'
df.groupby('residual_sugar').quality.mean()

residual_sugar
high    5.665880
low     5.602394
Name: quality, dtype: float64

In [18]:
median_citric_acid = df.citric_acid.median()
for i, citric_acid in enumerate(df.citric_acid):
    if citric_acid >= median_citric_acid:
        df.loc[i, 'citric_acid'] = 'high'
    else:
        df.loc[i, 'citric_acid'] = 'low'
df.groupby('citric_acid').quality.mean()

citric_acid
high    5.822360
low     5.447103
Name: quality, dtype: float64

Por fim, aqui está uma função que nos ajuda a criar um código mais conciso e limpo para fazer essa breve análise.

In [19]:
def features_to_analyse(df, feature):
    '''
    Função para analisar as fetures calculando a média e a proporção de valores maiores e menores que a média.
    '''
    median_column_value = df[feature].median()
    # Para cada valor em df[feature]
    for i, j in enumerate(df[feature]):
        if j >= median_column_value:
            # Substitui todos os valores da condição por 'high'
            df.loc[i, feature] = "high"
        else:
            # Substitui todos os valores da condição por 'low'
            df.loc[i, feature] = "low"

In [20]:
columns_to_analyse = ['pH', 'alcohol', 'residual_sugar', 'citric_acid']

for each in columns_to_analyse:
    features_to_analyse(data, each)
    print(data.groupby(each).quality.mean(), '/n')

pH
high    5.598039
low     5.675607
Name: quality, dtype: float64 /n
alcohol
high    5.958904
low     5.310302
Name: quality, dtype: float64 /n
residual_sugar
high    5.665880
low     5.602394
Name: quality, dtype: float64 /n
citric_acid
high    5.822360
low     5.447103
Name: quality, dtype: float64 /n
